In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from google.colab import drive
drive.mount('/content/drive')
# device='cuda' if torch.cuda.is_available() else 'cpu'
# print(torch.cuda.get_device_name(0))

# filenames=['./drive/My Drive/gnn/ieee24_rts_6183exact_0810.txt', # exact nodal injection (generator included)
#            './drive/My Drive/gnn/data/ieee118_dc_7859_price_lambda.txt', # load file (no gen)
#            './drive/My Drive/gnn/data/ieee118_dc_8000_cost2price.txt',
#            './drive/My Drive/gnn/data/ieee118_dc_8000_3input2price.txt',
#            '/content/drive/MyDrive/gnn/data/ieee118_dc_8000_3input2price_new.txt'
#            '/content/drive/MyDrive/gnn/data/ieee118_dc_10000_3input2price.txt',
#            '/content/drive/MyDrive/gnn/data/ieee118_dc_10000_3input2price_p10.txt',
#            '/content/drive/MyDrive/gnn/data/ieee118_ac_20000_3input2price_p10_1.txt']
filename=filenames[-1]

Mounted at /content/drive


In [ ]:
data=pd.read_table(filename,sep=',',header=None).to_numpy()
## load injection
# system size
n_bus=int(data[0,0].copy())
n_line=int(data[1,0].copy()) # not concatenated
n_line=179
n_load=int(data[2,0].copy())
n_sample=int(data[3,0].copy())
# line
line_bus=data[:,1:3].copy()
# load data
load_data0=data[0:n_bus,3:n_sample+3].copy()
# generator cost data
gen_cost0=data[0:n_bus,n_sample+3:2*n_sample+3].copy()
# generator limit data
gen_limit0=data[0:n_bus,2*n_sample+3:3*n_sample+3].copy()
# LMP data
lmp_data=data[0:n_bus,3*n_sample + 3 : 4*n_sample + 3].copy()
# generation data
gen_data=data[0:n_bus,4*n_sample + 3 : 5*n_sample + 3].copy()
y0=lmp_data.copy()
print(n_sample)
print(np.max(gen_cost0),np.min(gen_cost0))
# Use partial data for faster test
n_sample=n_sample
load_data=np.zeros((n_bus,3,n_sample))

load_data[:,0,:]=load_data0[:,:n_sample].copy()
load_data[:,1,:]=gen_cost0[:,:n_sample].copy()
load_data[:,2,:]=gen_limit0[:,:n_sample].copy()
y0=y0[:,:n_sample].copy()*1
# print system info
print('Test case: ',filename)
print('Number of buses: ',n_bus)
print('Number of lines: ',n_line)
print('Number of loads: ',n_load)
print('Number of samples: ',n_sample)

20000
149.5 0.0
Test case:  /content/drive/MyDrive/gnn/data/ieee118_ac_20000_3input2price_p10_1.txt
Number of buses:  118
Number of lines:  179
Number of loads:  99
Number of samples:  20000


In [ ]:
# filename = './drive/My Drive/gnn/data/ieee118_nominal_price.txt' # load file (no gen)
filename='./drive/My Drive/gnn/data/ieee118_nominal_cost2price.txt'
nominal_data=pd.read_table(filename,sep=',',header=None).to_numpy() # range [-0.4,0.5]
load_nominal=nominal_data[:,0].copy()
lmp_nominal=nominal_data[:,1].copy()
for i in range(n_sample):
  # load_data[:,i] = load_data[:,i] - load_nominal
  y0[:,i] = y0[:,i] #- lmp_nominal
  load_data[:,:,i] = load_data[:,:,i] #- load_nominal  #/ 400
# y0 = y0 / 1500
# load_data = np.log(load_data+1) # +1 to avoid -inf in load, scaling / 300
# y0 = np.log(y0) 
print(load_data.shape,y0.shape)
# print(load_data[:,1:5],y0[:,1:5])
print(np.max(load_data),np.min(load_data),np.max(y0),np.min(y0),max(lmp_nominal),min(lmp_nominal))

(118, 3, 20000) (118, 20000)
1418.4 0.0 350.92 10.091000000000001 3396.4 3370.5


In [ ]:
filename='./drive/My Drive/gnn/data/ieee118_Bbus.txt' # load file (no gen)
B=pd.read_table(filename,sep=',',header=None).to_numpy() # range [-0.4,0.5]
print(B.shape,type(B))

line_src=line_bus[:,0].copy().astype(int)
line_dst=line_bus[:,1].copy().astype(int)
print('Original graph:')
print('There are %d nodes.' % n_bus)
print('There are %d edges.' % n_line)

# adjacency
def graph_v2v_mat(line,n_line,n_bus):
	w = np.zeros((n_bus,n_bus))
	for i in range(n_line):
		# print(int(line[i,0]),int(line[i,1]))
		w[int(line[i,0])-1,int(line[i,1])-1] = 1
		w[int(line[i,1])-1,int(line[i,0])-1] = 1
	return w

def graph_laplacian(line,n_line,n_bus):
	incidence = np.zeros((n_line,n_bus))
	for i in range(n_line):
		# print(int(line[i,0]),int(line[i,1]))
		incidence[i,int(line[i,0])-1] = incidence[i,int(line[i,0])-1] + 1 # source
		incidence[i,int(line[i,1])-1] = incidence[i,int(line[i,1])-1] - 1 # destination
	w = np.matmul(incidence.transpose(),incidence)
	return w
W = graph_laplacian(line_bus,n_line,n_bus) # v2v
A = graph_v2v_mat(line_bus,n_line,n_bus)
# W=np.abs(W)

(118, 118) <class 'numpy.ndarray'>
Original graph:
There are 118 nodes.
There are 179 edges.


In [ ]:
print(W)
w1 = np.matmul(W,W)
a1 = np.matmul(A,A)
print(W[1,:])
print(np.sum(w1[1,:]),w1[1,:])
print(A[1,:])
print(a1[1,:])

[[ 2. -1. -1. ...  0.  0.  0.]
 [-1.  2.  0. ...  0.  0.  0.]
 [-1.  0.  3. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  0.  1.  0.]
 [ 0.  0.  0. ...  0.  0.  2.]]
[-1.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0.0 [-4.  6.  2.  0.  0.  0.  1.  0.  0.  0.  1. -9.  0.  1.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.

In [ ]:
# Set the ratio of training/test set
train_ratio = 0.8
n_train = int(np.floor(train_ratio * n_sample))
n_test = n_sample - n_train
print('Data set size: ',n_sample,', training set size: ',n_train,', test set size: ',n_test)

# Generate training set and test set
from sklearn.model_selection import train_test_split
x_train2, x_test2, y_train2, y_test2 = train_test_split(load_data[0:n_bus,:].transpose(), y0[:,0:n_sample].transpose(), test_size=0.2, random_state=23)

print(x_train2.shape, y_train2.shape)
x_train=x_train2.transpose()
print(x_train.shape)
x_test=x_test2.transpose()
y_train=y_train2.transpose()
y_test=y_test2.transpose()
x_all=load_data[0:n_bus,:]
y_all=y0[:,0:n_sample]

print('Training data size:',x_train.shape)
print('Training label size:',y_train.shape)

def get_xy(index):
    tempx=np.array([np.transpose(x_train)[index]])
    x_train_one=torch.from_numpy(tempx).float()
    x_train_one=x_train_one.transpose(0,1)

    tempy=np.array([np.transpose(y_train)[index]])
    y_train_one=torch.from_numpy(tempy).float()
    y_train_one=y_train_one.transpose(0,1)

    return x_train_one,y_train_one

# generate torch y_train for concatenated obj
y_train1 = torch.from_numpy(y_train)#.float()
# expand into 3d tensor 
y_train1.unsqueeze_(-1)
y_train1 = y_train1.expand(n_bus,n_train,1)
y_train1 = y_train1.transpose(2,1)
print(y_train1.shape)

Data set size:  20000 , training set size:  16000 , test set size:  4000
(16000, 3, 118) (16000, 118)
(118, 3, 16000)
Training data size: (118, 3, 16000)
Training label size: (118, 16000)
torch.Size([118, 1, 16000])


In [ ]:
# AC-OPF
np.save('/content/drive/MyDrive/gnn/data/ac118_p10_x.npy',arr=x_all)
np.save('/content/drive/MyDrive/gnn/data/ac118_p10_y.npy',arr=y_all)
np.save('/content/drive/MyDrive/gnn/data/ac118_p10_w.npy',arr=W)

# # DC-OPF
# np.save('/content/drive/MyDrive/gnn/data/dc118_p10_x.npy',arr=x_all)
# np.save('/content/drive/MyDrive/gnn/data/dc118_p10_y.npy',arr=y_all)
# np.save('/content/drive/MyDrive/gnn/data/dc118_p10_w.npy',arr=W)